**Notebook for the pre-processing of the Sirris Wind Turbine data**

-The data can be downloaded at https://opendata-renewables.engie.com/explore/index

-Only the data files are needed (la-haute-borne-data-2013-2016.csv and la-haute-borne-data-2017-2020.csv)

-Change the block below to the correct path of the folder that contains the downloaded data.

-There are 7 steps in the pre-processing. Steps 3-6 are optional.

In [ ]:
#put the correct path to the data folder here
data_path = 'C:/Users/adloose/Downloads/WindTurbines_Sirris'

**Step 0: load packages and define functions**

In [ ]:
#necessary packages
import os
import math
import numpy as np
import pandas as pd

In [ ]:
#necessary functions
def AD_imputeValues(data):
    a=0
    a_flag=True
    min_val=0
    max_val=0
    for i in range(data.shape[0]):
        check=np.sum(np.isnan(data[i, :]))
        if check>0: #nan found
            a=1 #at least 1 value is missing
            a_flag=True
            while a_flag:
                if np.sum(np.isnan(data[i+a, :]))>0: #next point is also missing, continue checking
                    a+=1 #one extra value missing
                else: #next point exists, end while loop
                    a_flag=False
            for k in range(a+4): #for all missing values in a row
                min_val=data[i-4, :] #just in case, go four values back
                max_val=data[i+a, :] #i+a is the first valid value (right after the nan value)
                #new value is:
                data[i-4+k, :]=min_val+((k+1)*(max_val-min_val)/(a+5)) #start overwriting from i-4
                #for 2 missing values, a=2 and k=0 and 1
                #which means i+1 is min_val+(1/3)*(max_val-min_val)
                #and i+2 is min_val+(2/3)*(max_val-min_val)
    return(data)

#interpolation function
def AD_interpolate(num_points, data, min_index):
    data_out=np.zeros((num_points, data.shape[1]),)
    for i in range(num_points):
        data_out[i, :]=data[min_index, :]+(i+1)*(data[min_index+1, :]-data[min_index, :])/(num_points+1)
        #data is missing, so min_index is point before and max_index is the point right after that
        #starting point: data[min_index, :]
        #slope: (data[min_index+1, :]-data[min_index, :])/(num_points+1)
        #location between start and stop: i+1
        #for two points the result is: start point + 1/3(end_point-start_point) and start point + 2/3(end_point-start_point)
    return(data_out)

#double timestamp function
def AD_doublestamp(data, s, t):
    #s starting value, t: time
    impute_data=np.zeros((t, data.shape[1]),)
    for i in range(0, t*2, 2):
        if math.isnan(data[s+i, 0]): #first value is missing
            impute_data[i//2, :]=data[s+i+1, :] #put in second value
        elif math.isnan(data[s+i+1, 0]): #second value is missing
            impute_data[i//2, :]=data[s+i, :] #put in first value
        else: #neither value is missing
            impute_data[i//2, :]=data[s+i+1, :] #put in first value
    data_out=np.concatenate((data[0:s, :], impute_data, data[(s+t*2):, :]), axis=0)
    return(data_out)

#triangle wave functions
def s_func(x, offset):
    x+=offset #make sure all angles start at 0 degrees
    y=np.zeros((len(x),))
    for i in range(len(x)):
        if x[i]<=90: #0-90
            y[i]=x[i]/90
        elif x[i]>=270: #270-360
            y[i]=(x[i]-360)/90
        else: #90-270
            y[i]=(180-x[i])/90    
    return(y)

def c_func(x, offset):
    y=s_func(x, offset-90)
    return(y)

**Step 1: read in data and select colums**

-The files are read, transformed to np arrays and joined together.

-A second wind vane was installed in 2017. To keep the variables consistent, the data from these two windvanes is averaged into one variable.

-Wind turbine and timestamp are saved into a seperate list. The remaining data is now numerical

-A list of remaining variables is kept (original list: var_list1, new list: var_list2)

In [ ]:
#read in all data
data_1316 = pd.read_csv(data_path+'/la-haute-borne-data-2013-2016.csv', sep=';')
var_list1=list(data_1316.columns.values) #list of all the variables
data_1316 = data_1316.values #numpy array

data_1720 = pd.read_csv(data_path+'/la-haute-borne-data-2017-2020.csv', sep=';')
data_1720 = data_1720.values #numpy array

#fill in Va colums in 1720 (second wind vane is installed in 2017)
for i in range(data_1720.shape[0]):
    if math.isnan(data_1720[i, 98]) and not math.isnan(data_1720[i, 90]): #if Va is missing and Va1 is present
        for j in range(4):
            data_1720[i, 98+j]=np.mean((data_1720[i, 90+j], data_1720[i, 94+j]), axis=0)

#join 2013-2016 together with 2017-2020
data_full=np.concatenate((data_1316, data_1720), axis=0)


#turbine and timestamp
turbine=data_full[:,0]
timestamps=data_full[:,1]
#extract useful part of timestamp
for i in range(len(timestamps)):
    timestamps[i]=timestamps[i][0:16]

#convert to lists
turbine=list(turbine)
timestamps=list(timestamps)

In [ ]:
#select used data colums
data=np.concatenate((data_full[:, 2:90], data_full[:, 98:126]), axis=1).astype('float64')
#leave out 0 and 1 (time and turbine), 90-98 (windvane 1 and 2), and beyond 126 (empty)
#make list variables
var_list2=var_list1[2:90]+var_list1[98:126]

**Step 2: chronological ordering**

-The data is ordered chronologically and stored into arrays. The arrays are:

data_WT1: turbine R80736

WT2: R80721

WT3: R80711

WT4: R80790

In [ ]:
#put everything in chronological order

#pre-allocate a set of arrays. Keep in mind some timestamps could be missing, so we will make the arrays large enough.
data_WT1=np.zeros((300000, 116), dtype='float64')
data_WT2=np.zeros((300000, 116), dtype='float64')
data_WT3=np.zeros((300000, 116), dtype='float64')
data_WT4=np.zeros((300000, 116), dtype='float64')

#we will count how many datapoints each wind turbine has.
count1=0 #init counter
count2=0 #init counter
count3=0 #init counter
count4=0 #init counter
dummy_index=0

timestamps_indexes=np.argsort(timestamps) #list of indexes in chronological order

#go through every timestamp and allocate it
for i in range(len(timestamps_indexes)):
    dummy_index=timestamps_indexes[i] #correct index
    #check which turbine it matches with (using the turbine list)
    if turbine[dummy_index]=='R80736':
        data_WT1[count1, :]=data[dummy_index, :]
        count1+=1 #add 1 to counter
    elif turbine[dummy_index]=='R80721':
        data_WT2[count2, :]=data[dummy_index, :]
        count2+=1
    elif turbine[dummy_index]=='R80711':
        data_WT3[count3, :]=data[dummy_index, :]
        count3+=1
    elif turbine[dummy_index]=='R80790':
        data_WT4[count4, :]=data[dummy_index, :]
        count4+=1

In [ ]:
#cut off extra zeros using counters
data_WT1=data_WT1[0:count1, :]
data_WT2=data_WT2[0:count2, :]
data_WT3=data_WT3[0:count3, :]
data_WT4=data_WT4[0:count4, :]

The remaining steps will all overwrite these four data arrays, so any of them can be skipped or adapted. To write out the raw data from here, skip to step 7.

**Step 3: imputing missing timestamps**

-The missing timestamps were located manually (the chronological list can be checked). We found that 2 days and one extra timestamp are missing. These are interpolated lineary using the function that was defined earlier (interpolate function).

-There is one set of double timestamps in WT1-WT3. These are dealt with using the function defined earlier.

In [ ]:
#impute missing timestamps

#missing timestamps were located manually

#custom function
def AD_joining(data_WT):
    data_20130127=AD_interpolate(1, data_WT, 3743)
    data_20130304=AD_interpolate(144, data_WT, 8928)
    #data_20130331=AD_interpolate(12, data_WT, 12826) #DST, not necessary
    data_20130514=AD_interpolate(144, data_WT, 19152)
    #data_20140330=AD_interpolate(12, data_WT, 64941)
    #data_20150329=AD_interpolate(12, data_WT, 117344)
    #data_20160327=AD_interpolate(12, data_WT, 169647)
    #data_20170326=AD_interpolate(12, data_WT, 222049)
    #data_20170610=AD_interpolate(144, data_WT, 232982) #double day, different function
    data_WT_out=np.concatenate((data_WT[0:3744, :], data_20130127, data_WT[3744:8929, :], data_20130304,
                            data_WT[8929:19153, :], data_20130514, data_WT[19153:, :]), axis=0)
    return(data_WT_out)

data_WT1=AD_joining(data_WT1)
data_WT2=AD_joining(data_WT2)
data_WT3=AD_joining(data_WT3)
data_WT4=AD_joining(data_WT4)

#fix the double timestamps mistake at 2017-06-10 in WT1-WT3
data_WT1=AD_doublestamp(data_WT1, 233424, 144)
data_WT2=AD_doublestamp(data_WT2, 233424, 144)
data_WT3=AD_doublestamp(data_WT3, 233424, 144)

**Step 4: impute empty variables**

-Empty variables are variables that have a nan value (which means the cell is present but is empty). This is not to be confused with the earlier missing timestamps, where a row is absent.

-Based on experience, it is found that one missing variable will mean the other variables are less reliable too, so these are interpolated (and hence overwritten) as well. The overwrite also starts 4 positions before the nan-value because often these datapoints right before a nan are unreliable too. This can be changed in the AD_imputeValues function defined earlier.

In [ ]:
#impute empty variables
data_WT1=AD_imputeValues(data_WT1)
data_WT2=AD_imputeValues(data_WT2)
data_WT3=AD_imputeValues(data_WT3)
data_WT4=AD_imputeValues(data_WT4)

**Step 5: angle conversions**

-Because degrees are hard to deal with, all angles are converted with triangular functions. This is similar to a sine and a cosine function. This creates some new variables and removes some old ones. The new set of variables is stored in var_list3

In [ ]:
#convert angles to triangle components

#variables to be converted:
list_c=list([0, 1, 2, 64, 65, 66, 84, 85, 86, 88, 89, 90])
offset_list=list([120, 120, 120, 0, 0, 0, 0, 0, 0, 180, 180, 180]) #Ba has offset 120 and Va has offset 180
var_angle_list=list()

#delete variables (useless angle_std components):
list_d=list([3, 67, 87, 91])

triangle_components=np.zeros((data_WT1.shape[0], len(list_c)*2),) #init

count=0
for i in range(len(list_c)):
    triangle_components[:, count]=s_func(data_WT1[:, list_c[i]], offset_list[i])
    triangle_components[:, count+1]=c_func(data_WT1[:, list_c[i]], offset_list[i])
    var_angle_list.append(var_list2[list_c[i]]+'_s')
    var_angle_list.append(var_list2[list_c[i]]+'_c')
    count+=2

var_list3=var_list2.copy()
list_del=list_c+list_d
list_del.sort()
for i in range(len(list_del)):
    var_list3.pop(list_del[i]-i) #delete components one by one, keep in mind index starts to shift
var_list3.extend(var_angle_list) #add new variables
    
#write out new variables in data array
data_WT1=np.concatenate((data_WT1[:, 4:64], data_WT1[:, 68:84], data_WT1[:, 92:], triangle_components), axis=1)

#repeat for WT2-4
#2
count=0
for i in range(len(list_c)):
    triangle_components[:, count]=s_func(data_WT2[:, list_c[i]], offset_list[i])
    triangle_components[:, count+1]=c_func(data_WT2[:, list_c[i]], offset_list[i])
    count+=2
data_WT2=np.concatenate((data_WT2[:, 4:64], data_WT2[:, 68:84], data_WT2[:, 92:], triangle_components), axis=1)

#3
count=0
for i in range(len(list_c)):
    triangle_components[:, count]=s_func(data_WT3[:, list_c[i]], offset_list[i])
    triangle_components[:, count+1]=c_func(data_WT3[:, list_c[i]], offset_list[i])
    count+=2
data_WT3=np.concatenate((data_WT3[:, 4:64], data_WT3[:, 68:84], data_WT3[:, 92:], triangle_components), axis=1)

#4
count=0
for i in range(len(list_c)):
    triangle_components[:, count]=s_func(data_WT4[:, list_c[i]], offset_list[i])
    triangle_components[:, count+1]=c_func(data_WT4[:, list_c[i]], offset_list[i])
    count+=2
data_WT4=np.concatenate((data_WT4[:, 4:64], data_WT4[:, 68:84], data_WT4[:, 92:], triangle_components), axis=1)

**Step 6: normalisation**

-All variables are normalised using the mean and the std. To make sure we do not divide by 0, we add 1e-5 to the std.

In [ ]:
#normalise variables

for i in range(data_WT1.shape[1]):
    data_WT1[:, i]=(data_WT1[:, i]-np.mean(data_WT1[:, i]))/(np.std(data_WT1[:, i])+1e-5)

for i in range(data_WT1.shape[1]):
    data_WT2[:, i]=(data_WT2[:, i]-np.mean(data_WT2[:, i]))/(np.std(data_WT2[:, i])+1e-5)

for i in range(data_WT1.shape[1]):
    data_WT3[:, i]=(data_WT3[:, i]-np.mean(data_WT3[:, i]))/(np.std(data_WT3[:, i])+1e-5)

for i in range(data_WT1.shape[1]):
    data_WT4[:, i]=(data_WT4[:, i]-np.mean(data_WT4[:, i]))/(np.std(data_WT4[:, i])+1e-5)

**Step 7: write out the data**

-The data is written out to the original folder as WT1_full-WT4_full.csv

-The variables are written out to a seperate csv file. Keep in mind this is a file without a header and var_list3 is used (the final set of variables). Earlier sets can be written out by changing this to var_list2 or var_list1.

In [ ]:
#write out the data per wind turbine in a csv file
np.savetxt(data_path+'/WT1_full.csv', data_WT1, delimiter=';')
np.savetxt('C:/Users/adloose/Downloads/WindTurbines_Sirris/WT2_full.csv', data_WT2, delimiter=';')
np.savetxt('C:/Users/adloose/Downloads/WindTurbines_Sirris/WT3_full.csv', data_WT3, delimiter=';')
np.savetxt('C:/Users/adloose/Downloads/WindTurbines_Sirris/WT4_full.csv', data_WT4, delimiter=';')
#list of variables
df = pd.DataFrame(data={"col1": var_list3})
df.to_csv('C:/Users/adloose/Downloads/WindTurbines_Sirris/var_list.csv', sep=';', index=False, header=False)